<a href="https://colab.research.google.com/github/PiotrusWatson/level4project/blob/master/data/ipynbs/textual_entailment_snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##HAHA ITS TIME TO SPEND 5 HOURS DOWNLOADING THINGS

lets get the snli dataset baybee

In [1]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip

--2020-02-06 17:21:29--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  26.5MB/s    in 3.6s    

2020-02-06 17:21:38 (25.0 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_

In [2]:
# Download the Glove.zip file and expand it.
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-02-06 17:21:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-06 17:21:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-06 17:21:47--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [3]:
# Get the PolitiFact Dataset from the location provided in the DeClarE paper.
!wget http://resources.mpi-inf.mpg.de/impact/dl_cred_analysis/PolitiFact.zip
!unzip PolitiFact.zip

--2020-02-06 17:28:38--  http://resources.mpi-inf.mpg.de/impact/dl_cred_analysis/PolitiFact.zip
Resolving resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)... 139.19.206.46
Connecting to resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)|139.19.206.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4976217 (4.7M) [application/zip]
Saving to: ‘PolitiFact.zip’

PolitiFact.zip      100%[===================>]   4.75M  5.45MB/s    in 0.9s    

2020-02-06 17:28:40 (5.45 MB/s) - ‘PolitiFact.zip’ saved [4976217/4976217]

Archive:  PolitiFact.zip
   creating: PolitiFact/
  inflating: PolitiFact/README       
  inflating: PolitiFact/politifact.tsv  


In [4]:
!git clone https://github.com/FakeNewsChallenge/fnc-1.git


Cloning into 'fnc-1'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49
Unpacking objects: 100% (49/49), done.


In [5]:
# Get the Snopes Dataset from the location provided in the DeClarE paper.
!wget http://resources.mpi-inf.mpg.de/impact/dl_cred_analysis/Snopes.zip
!unzip Snopes.zip

--2020-02-06 17:28:52--  http://resources.mpi-inf.mpg.de/impact/dl_cred_analysis/Snopes.zip
Resolving resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)... 139.19.206.46
Connecting to resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)|139.19.206.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5559754 (5.3M) [application/zip]
Saving to: ‘Snopes.zip’

Snopes.zip          100%[===================>]   5.30M  5.45MB/s    in 1.0s    

2020-02-06 17:28:53 (5.45 MB/s) - ‘Snopes.zip’ saved [5559754/5559754]

Archive:  Snopes.zip
   creating: Snopes/
  inflating: Snopes/README           
  inflating: Snopes/snopes.tsv       


Some imports lol :P

In [6]:
!pwd

/content


##LOOK AT ALL THIS CODE TO IMPORT DATA GOD THERE MUST BE SOMETHING WRONG WITH ME

In [7]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch,keras

from sklearn.model_selection import train_test_split, KFold
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.nn.parameter import Parameter
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data_utils

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

import math

np.random.seed(128)

Using TensorFlow backend.


lets load this shit :^)

In [0]:
train_dataframe = pd.read_json('./snli_1.0/snli_1.0_train.jsonl', lines=True)
test_dataframe = pd.read_json('./snli_1.0/snli_1.0_test.jsonl', lines=True)


In [9]:
train_dataframe.head(50)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
5,[contradiction],2267923837.jpg#2,contradiction,2267923837.jpg#2r1c,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,The kids are frowning,( ( The kids ) ( are frowning ) ),(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...
6,[contradiction],3691670743.jpg#0,contradiction,3691670743.jpg#0r1c,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy skates down the sidewalk.,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...
7,[entailment],3691670743.jpg#0,entailment,3691670743.jpg#0r1e,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy does a skateboarding trick.,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...
8,[neutral],3691670743.jpg#0,neutral,3691670743.jpg#0r1n,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy is wearing safety equipment.,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...
9,[neutral],4804607632.jpg#0,neutral,4804607632.jpg#0r1n,An older man sits with his orange juice at a s...,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man drinks his juice as he waits for ...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...


Helper functions: something that bulk converts things into lists, and a tokeniser that also pads and numpies things

In [10]:
def merge_bodies(articles, claims):
  merged = pd.merge(articles, claims, on="Body ID")
  mapping = {"disagree": 0, "discuss": 1, "unrelated": 2, "agree": 3}
  return merged.replace({"Stance": mapping})
  
  
train_articles = pd.read_csv("./fnc-1/train_bodies.csv")
train_claims = pd.read_csv("./fnc-1/train_stances.csv")
test_articles = pd.read_csv("./fnc-1/test_bodies.csv")
test_claims = pd.read_csv("./fnc-1/test_stances_unlabeled.csv")


train_challenge = merge_bodies(train_articles, train_claims)

test_challenge = merge_bodies(test_articles, test_claims)
train_challenge.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",2
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,2
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,2
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,2
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,2


also: lets load politifact :^^)

In [0]:
facts = pd.read_csv('./PolitiFact/politifact.tsv', delimiter = '\t', names = ['cred_label','claim_id','claim_text','claim_source','article','article_source'])
facts.head(50)
snopes = pd.read_csv("./Snopes/snopes.tsv", delimiter= "\t", names=['cred_label','claim_id','claim_text','article','article_source'])
politi_mapping = {"True": 1, "Half-True": 1, "Mostly True": 1, "Mostly False": 0, "False": 0, "Pants on Fire!": 0}
snopes_mapping = {"true": 1, "half-true": 1, "mostly true": 1, "mostly false": 0, "false": 0, "pants on fire!": 0}

def slice_snopes(unique):
  true_claims = unique[unique["cred_label"] == 1]
  false_claims = unique[unique["cred_label"] == 0]
  false_claims = false_claims.head(int(len(false_claims)/3))
  return pd.concat([true_claims, false_claims]).sample(frac=1)


def preprocess_fact_data(facts, mapping, slice_function=None, is_folding=False):
  
  facts = facts.replace({"cred_label": mapping})
  unique = facts.drop_duplicates("claim_id")
  if (slice_function):
    unique = slice_function(unique)
  
#splitting the claims
  
  if is_folding:
    results = []
    folded = KFold(n_splits=10, shuffle=True)
    splitted_object = folded.split(unique)
    for train_result, test_result in splitted_object:
      train_ilocs = unique.iloc[train_result]["claim_id"]
      test_ilocs = unique.iloc[test_result]["claim_id"]
      results.append((facts[facts["claim_id"].isin(train_ilocs)], facts[facts["claim_id"].isin(test_ilocs)]))

    return results

  train_unique, test_unique = train_test_split(unique, test_size=0.2, random_state=8)

  

#recreating dataset
  test_facts = facts[facts["claim_id"].isin(test_unique["claim_id"])]
  train_facts = facts[facts["claim_id"].isin(train_unique["claim_id"])]
  return train_facts, test_facts
#get unique claims to divide dataset cleanly
train_facts, test_facts = preprocess_fact_data(facts, politi_mapping)
train_snopes, test_snopes = preprocess_fact_data(snopes, snopes_mapping, slice_snopes)

folded_snopes = preprocess_fact_data(snopes, snopes_mapping, slice_snopes, True)
folded_fact = preprocess_fact_data(facts, politi_mapping, None, True)

In [12]:
test_facts.head(500)

,cred_label,claim_id,claim_text,claim_source,article,article_source
85,1,2014_aug_01_bruce-braley_iowa-senate-candidate...,says us senate candidate joni ernst not suppor...,bruce braley,morning new tv advertisement argues that posit...,desmoinesregister.com
86,1,2014_aug_01_bruce-braley_iowa-senate-candidate...,says us senate candidate joni ernst not suppor...,bruce braley,iowans could not support household on current ...,americanprogressaction.org
87,1,2014_aug_01_bruce-braley_iowa-senate-candidate...,says us senate candidate joni ernst not suppor...,bruce braley,not budged in five years leaving many falling ...,americanprogressaction.org
88,1,2014_aug_01_bruce-braley_iowa-senate-candidate...,says us senate candidate joni ernst not suppor...,bruce braley,are working to support themselves or their fam...,iowademocrats.org
89,1,2014_aug_01_bruce-braley_iowa-senate-candidate...,says us senate candidate joni ernst not suppor...,bruce braley,prove that i had those good hardworking skills...,iowademocrats.org
...,...,...,...,...,...,...
2766,0,2008_oct_02_joe-biden_mccain-refused-to-commit...,john mccain said wouldnt even sit government s...,joe biden,cap is melting palin im not one to attribute e...,mysinchew.com
2767,0,2008_oct_02_joe-biden_mccain-refused-to-commit...,john mccain said wouldnt even sit government s...,joe biden,control the weapons the theocracy does secreta...,blastmagazine.com
2768,0,2008_oct_02_joe-biden_mccain-refused-to-commit...,john mccain said wouldnt even sit government s...,joe biden,be left with only one conclusion mccain was co...,chrisweigant.com
2850,0,2009_may_19_mike-pence_120-million-deprived-he...,democrats propose health care plan deprive rou...,mike pence,speech politifact called claim false that demo...,democrats.org


In [0]:
def convert_to_lists(names_to_lists):
  for key in names_to_lists:
    names_to_lists[key] = names_to_lists[key].tolist()
  return names_to_lists

class Tokeniser:
  def __init__(self, texts, vocab_size, max_len):
    self.t = Tokenizer()
    self.max_len = max_len
    self.t.num_words = vocab_size
    
    full_corpus = []

    for index in texts:
      for text in texts[index]:
        full_corpus.append(text)
    
    self.t.fit_on_texts(full_corpus)

  def full_process(self, text):
    """OK SO: converts a list of strings into a list of numerical sequences
then pads them out so they're all a consistent size
then returns a numpy array of that :) """
    new_sequence = self.t.texts_to_sequences(text)
    #todo: modify to make it spit out a summarised version ABOUT HERE
    padded_sequence = pad_sequences(new_sequence, maxlen=self.max_len, padding ="post")
    return np.array(padded_sequence, dtype=np.float32)

  def do_everything(self, texts):
    for index in texts:
      texts[index] = self.full_process(texts[index])
    self.word_to_id = self.t.word_index
    return texts



  
# Get the embedding matrix using Glove. 
vocab,word2idx = None,{}

def load_glove_embeddings(path, word2idx, embedding_dim):
    """Loading the glove embeddings"""
    vocab_size = len(word2idx) + 1
    print(vocab_size)
    with open(path) as f:
        embeddings = np.zeros((vocab_size, embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = word2idx.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                if vector.shape[-1] != embedding_dim:
                    raise Exception('Dimension not matching.')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

#assumption: we're going to only care about classification per text
def generate_indexes(labels):
  return [1 if label == "neutral" else 2 if label == "entailment" else 0 for label in labels]

index_to_label = ["contradiction","neutral","entailment"]


In [0]:
class Dataset:
  def __init__(self, train_loader, test_loader, test_data, tokeniser):
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.test_data = test_data
    self.word_embeddings_small = load_glove_embeddings("glove.6B.50d.txt", tokeniser.word_to_id, 50) 

In [0]:
labels = ["claim_text", "article"]
big_labels = ["claim_text", "article", "article_source"]
def get_dataset(train, test, vocab_size, max_length, batch_size, labels, name):
  train_list = {}
  test_list = {}
  #converting array into dict for EVERYTHING ELSE TO WORK
  for label in labels:
    train_list[label] = train[label]
    test_list[label] = test[label]

  #LETS GET SOME LISTS INSTEAD OF PANDAS SERIES'
  train_list_x = convert_to_lists(train_list)
  test_list_x = convert_to_lists(test_list)
  train_list_y = train["cred_label"].tolist()
  test_list_y = test["cred_label"].tolist()

  #tokenising various stuff, setting up numpy dictionaries :)
  tokeniser = Tokeniser(train_list_x, vocab_size, max_length)
  x_train = tokeniser.do_everything(train_list_x)
  x_test = tokeniser.do_everything(test_list_x)
  y_train = np.array(train_list_y, dtype=np.float32)
  y_test = np.array(test_list_y, dtype=np.float32)
  
  #datasets/loaders
  training_stuff = []
  testing_stuff = []
  for key in x_train:
    training_stuff.append(torch.from_numpy(x_train[key]).type(torch.LongTensor))
    testing_stuff.append(torch.from_numpy(x_test[key]).type(torch.LongTensor))
  training_stuff.append(torch.from_numpy(y_train).type(torch.DoubleTensor))
  testing_stuff.append(torch.from_numpy(y_test).type(torch.DoubleTensor))
  train_dataset = data_utils.TensorDataset(*training_stuff)
  test_dataset = data_utils.TensorDataset(*testing_stuff)
  train_loader = data_utils.DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True)
  test_loader = data_utils.DataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=False)
  train_loader.name = name
  test_loader.name = name
  return Dataset(train_loader, test_loader, y_test, tokeniser)





here i set up the tokeniser, and turn everything into a list its a fun cell

In [16]:
MAX_LENGTH = 500
VOCAB_SIZE = 20000
BATCH_SIZE = 256
SAMPLE_SAMPLE_SIZE = 1


snopes_dataset = get_dataset(train_snopes, test_snopes, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE, labels, "fact_data")
fact_dataset = get_dataset(train_facts, test_facts, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE, labels, "fact_data")
big_snopes = get_dataset(train_snopes, test_snopes, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE, big_labels, "fact_data")
big_fact = get_dataset(train_facts, test_facts, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE, big_labels, "fact_data")

snopes_folded_datasets = []
fact_folded_datasets = []

for train, test in folded_snopes:
  snopes_folded_datasets.append(get_dataset(train, test, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE, labels, "fact_data"))

for train, test in folded_fact:
  fact_folded_datasets.append(get_dataset(train, test, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE, labels, "fact_data"))

chopped_train_dataframe = train_dataframe.sample(n=int(len(train_dataframe["sentence1"])/SAMPLE_SAMPLE_SIZE))
x_train_lists = convert_to_lists({"premise": chopped_train_dataframe["sentence1"], "hypothesis": chopped_train_dataframe["sentence2"]})
y_train_list = chopped_train_dataframe["gold_label"].tolist()

x_test_lists = convert_to_lists({"premise": test_dataframe["sentence1"], "hypothesis": test_dataframe["sentence2"]})
y_test_list = test_dataframe["gold_label"].tolist()

x_train_challenge_list = convert_to_lists({"claim_text": train_challenge["Headline"], "article": train_challenge["articleBody"]})
y_train_challenge_list = train_challenge["Stance"].tolist()

x_test_challenge_list = convert_to_lists({"claim_text": test_challenge["Headline"], "article": test_challenge["articleBody"]})





39181
33494
44784
36848
41475
41387
41246
41527
41597
41669
41427
41549
41540
41321
34917
35116
34977
35015
35178
35181
35161
35135
35150
35247


this cell uses the setup tokeniser to SLAP THAT SHIT INTO NUMPY ARRAYS WITH PADDING YEAH BABY
(also tokenises it thats p important)

In [0]:
x_tokeniser = Tokeniser(x_train_lists, VOCAB_SIZE, MAX_LENGTH)
challenge_tokeniser = Tokeniser(x_train_challenge_list, VOCAB_SIZE, MAX_LENGTH)

x_train = x_tokeniser.do_everything(x_train_lists)
x_test = x_tokeniser.do_everything(x_test_lists)
y_train = np.array(generate_indexes(y_train_list), dtype=np.float32)
y_test = np.array(generate_indexes(y_test_list), dtype=np.float32)

x_challenge_train = challenge_tokeniser.do_everything(x_train_challenge_list)
x_challenge_test = challenge_tokeniser.do_everything(x_test_challenge_list)
y_challenge_train = np.array(y_train_challenge_list, dtype=np.float32)



and here we slap the loaded stuff into a neat tensordataset. this is good because ???

In [0]:
we_shufflin = True
shufflin_test = False
#alright lets tensordataset textual entailment stuff
train_data = data_utils.TensorDataset(torch.from_numpy(x_train["premise"]).type(torch.LongTensor),
                                      torch.from_numpy(x_train["hypothesis"]).type(torch.LongTensor),
                                      torch.from_numpy(y_train).type(torch.DoubleTensor))
train_loader = data_utils.DataLoader(train_data, batch_size=BATCH_SIZE, drop_last=True, shuffle=we_shufflin)
train_loader.name = "entailment_data"

test_data = data_utils.TensorDataset(torch.from_numpy(x_test["premise"]).type(torch.LongTensor),
                                      torch.from_numpy(x_test["hypothesis"]).type(torch.LongTensor),
                                      torch.from_numpy(y_test).type(torch.DoubleTensor))
test_loader = data_utils.DataLoader(test_data, batch_size=BATCH_SIZE, drop_last=False, shuffle=shufflin_test )
test_loader.name = "entailment_data"


#POLITIFACT/SNOPES W/ SOURCES


train_data = data_utils.TensorDataset(torch.from_numpy(x_challenge_train["claim_text"]).type(torch.LongTensor),
                                      torch.from_numpy(x_challenge_train["article"]).type(torch.LongTensor),
                                      torch.from_numpy(y_challenge_train).type(torch.DoubleTensor))
train_challenge_loader = data_utils.DataLoader(train_data, batch_size=BATCH_SIZE, drop_last=True, shuffle=we_shufflin)
train_challenge_loader.name = "challenge_data"

test_data = data_utils.TensorDataset(torch.from_numpy(x_challenge_test["claim_text"]).type(torch.LongTensor),
                                      torch.from_numpy(x_challenge_test["article"]).type(torch.LongTensor))
test_challenge_loader = data_utils.DataLoader(test_data, batch_size=BATCH_SIZE, drop_last=False, shuffle=shufflin_test )
test_loader.name = "challenge_data"

Helper function. I don't know why we have such a helper function but it's here.
Does a softmax after transposing and reshaping things ??


In [0]:
def softmax(input, axis=1):
    """
        Softmax applied to axis=n
 
        Args:
           input: {Tensor,Variable} input on which softmax is to be applied
           axis : {int} axis on which softmax is to be applied
 
        Returns:
            softmaxed tensors
 
       
    """
    input_size = input.size()
    trans_input = input.transpose(axis, len(input_size)-1)
    trans_size = trans_input.size()
    input_2d = trans_input.contiguous().view(-1, trans_size[-1])
    soft_max_2d = F.softmax(input_2d)
    soft_max_nd = soft_max_2d.view(*trans_size)  
    return soft_max_nd.transpose(axis, len(input_size)-1)

First part of the model (split out so to test alone)
Basically, a wrapper for an lstm
Takes in a sequence, spits out a sequence of matrices demonstrating ~an understanding~ of the data

##TEXTUAL ENTAILMENT MODEL CODE

In [0]:
class SequenceProcessor(torch.nn.Module):  
  def __init__(self, word_embeddings, hp):
    super(SequenceProcessor, self).__init__()
    self.hp = hp
    self.embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.embedding_size = word_embeddings.size(1)
    self.cool_lstm = torch.nn.LSTM(
        input_size = self.embedding_size,
        hidden_size = hp.lstm_hidden_size,
        num_layers=1,
        batch_first=True,
        bidirectional=True
      )

    
  def forward(self, x, hidden_layer):
    embedding = self.embeddings(x)
    return self.cool_lstm(embedding,
                          hidden_layer)


Next bit of model. Given a processed set of 

In [0]:
class AttentionModel(torch.nn.Module):
  def __init__(self, hp):
    super(AttentionModel, self).__init__()
    self.first_linear = torch.nn.Linear(
        in_features= 2*hp.lstm_hidden_size,
        out_features = hp.dense_dimension,
        bias = False
    )
    self.second_linear = torch.nn.Linear(
        in_features = hp.dense_dimension,
        out_features = hp.attention_hops,
        bias = False
    )
    self.dropout = torch.nn.Dropout(p=hp.dropout)

  def forward(self, x):
    x = self.dropout(x)
    tanh_W_H = torch.tanh(self.first_linear(x))
    #[512 rows, 150 numerical words, of size 100] (512, 150, 100) <bmm> (1, 100, 100) = (512, 150, 100)
    #another batch matrix multiply, wow!
    weight_by_attention_hops = self.second_linear(tanh_W_H) # (100, 10) by (512, 10, 100)
    #[512 rows, 10 attention hops of size 100] (512, 150, 100) <bmm> (1, 10, 100) = (512, 10, 150)
    
    attention = softmax(weight_by_attention_hops).transpose(2,1)
    sentence_embeddings = torch.bmm(attention,x)
    return sentence_embeddings, attention


In [0]:
def better_mush(premise, hypothesis):
    pooled_premise1 = premise[:,:,::2]
    pooled_premise2 = premise[:,:,1::2]
    pooled_hypothesis1 = hypothesis[:,:,::2]
    pooled_hypothesis2 = hypothesis[:,:,1::2]

    better_mush = torch.cat((pooled_premise1 * pooled_hypothesis1 + pooled_premise2 * pooled_hypothesis2,
                               pooled_premise1 * pooled_hypothesis2 - pooled_premise2 * pooled_hypothesis1),2)
    return better_mush

class Factoriser(torch.nn.Module):
  def __init__(self, hp):
    super(Factoriser, self).__init__()
    self.premise_weight = Parameter(torch.Tensor(
        hp.attention_hops, 
        hp.lstm_hidden_size*2,
        hp.gravity
        ))
    self.hypothesis_weight = Parameter(torch.Tensor(
        hp.attention_hops, 
        hp.lstm_hidden_size*2,
        hp.gravity
        ))
    init.kaiming_uniform_(self.premise_weight, a=math.sqrt(5))
    init.kaiming_uniform_(self.hypothesis_weight, a=math.sqrt(5))

  def batcheddot(self, a, b):
    better_a = a.transpose(0,1)
    bmmd = torch.bmm(better_a, b)
    return bmmd.transpose(0,1)

  def forward(self, premise, hypothesis):

    premise_factor = self.batcheddot(premise, self.premise_weight)
    hypothesis_factor = self.batcheddot(hypothesis, self.hypothesis_weight)
    return better_mush(premise_factor,hypothesis_factor)

In [0]:
class MLP(torch.nn.Module):
  def __init__(self, hp):
    super(MLP, self).__init__()
    self.linear1 = torch.nn.Linear(
        in_features=hp.attention_hops*hp.gravity, 
        out_features=20)
    if hp.avg:
      self.final_linear = torch.nn.Linear(hp.gravity, hp.num_classes)
    else:
      self.final_linear = torch.nn.Linear(20, hp.num_classes)
    self.hp = hp
  def forward(self, x):
    if self.hp.avg:
      x = torch.sum(x, 1)/self.hp.attention_hops
    else:
      x = self.linear1(x.reshape(self.hp.batch_size, -1))
    if (self.hp.num_classes > 1):
      x = softmax(self.final_linear(x))
    else:
      x = torch.sigmoid(self.final_linear(x))
    return x

In [0]:
class TextualEntailmentModel(torch.nn.Module):

  def init_hidden(self):
    hidden_state = torch.zeros(2,self.hp.batch_size,self.hp.lstm_hidden_size).cuda()
    cell_state = torch.zeros(2,self.hp.batch_size, self.hp.lstm_hidden_size).cuda()
    return (hidden_state, cell_state)
  def reset_for_testing(self, new_batch):
    self.hp.batch_size = new_batch
    self.hidden_state = self.init_hidden()

  def __init__(self, hp, word_embeddings):
    super(TextualEntailmentModel, self).__init__()
    self.hp = hp
    self.premise_processor = SequenceProcessor(word_embeddings, hp)
    self.hypothesis_processor = SequenceProcessor(word_embeddings, hp)
    self.premise_embedder = AttentionModel(hp)
    self.hypothesis_embedder = AttentionModel(hp)
    self.factoriser = Factoriser(hp)
    self.MLP = MLP(hp)
    self.hidden_state = self.init_hidden()
  
  def forward(self, premise, hypothesis):
    processed_premise, self.hidden_state = self.premise_processor(premise, self.hidden_state)
    premise_embedding, premise_attention = self.premise_embedder(processed_premise)
    processed_hypothesis, self.hidden_state = self.hypothesis_processor(hypothesis, self.hidden_state)
    hypothesis_embedding, hypothesis_attention = self.hypothesis_embedder(processed_hypothesis)
    factorised_mush = self.factoriser(premise_embedding, hypothesis_embedding)
    return self.MLP(factorised_mush), hypothesis_attention



##EVAL SUMMARY :)

In [0]:
def evaluation_summary(description, predictions, unnormalised_predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels, average='macro')
  recall = recall_score(predictions, true_labels, average='macro')
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1, average='macro') #1 means f_1 measure
  if (len(unnormalised_predictions.shape) == 1):
    fpr, tpr, thresholds = roc_curve(true_labels, predictions)
    auc_full = auc(fpr, tpr)
  else:
    auc_full = 0
  print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f AUC=%0.3f" % (description,accuracy,precision,recall,f1, auc_full))
  print(classification_report(predictions, true_labels, digits=3))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))
  return precision,recall,accuracy,f1 

##SHEENABASELINE CODE

In [0]:
class BaselineSentenceEntailment(torch.nn.Module):
  def init_hidden(self):
    hidden_state = Variable(torch.zeros(2,self.hp.batch_size,self.hp.lstm_hidden_size)).cuda()
    cell_state = Variable(torch.zeros(2,self.hp.batch_size, self.hp.lstm_hidden_size)).cuda()
    return (hidden_state, cell_state)
  def reset_for_testing(self, new_batch):
    self.hp.batch_size = new_batch
    self.hidden_state = self.init_hidden()

  def __init__(self, hp, word_embeddings):
    super(BaselineSentenceEntailment, self).__init__()
    self.hp = hp
    self.premise_embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.premise_embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.premise_embedding_size = word_embeddings.size(1)
    self.premise_lstm = torch.nn.LSTM(
        input_size = self.premise_embedding_size,
        hidden_size = hp.lstm_hidden_size,
        num_layers=1,
        batch_first=True,
        bidirectional=True
      )
    self.hypothesis_processor = SequenceProcessor(word_embeddings, hp)
    self.premise_embedder = AttentionModel(hp)
    self.hypothesis_embedder = AttentionModel(hp)
    self.linear_final = torch.nn.Linear(hp.lstm_hidden_size*2, hp.num_classes)

  def forward(self, premise, hypothesis):
    #premise/hypothesis embeddinbgs
    embeddings = self.premise_embeddings(premise)
    added_embeddings = self.premise_embeddings(hypothesis[:, :100])
    main_embeddings = torch.cat((embeddings, added_embeddings), 1)
    reshaped_embeddings = main_embeddings.view(self.hp.batch_size, self.hp.max_length, -1)
    processed_premise, hidden_state = self.premise_lstm(main_embeddings, self.hidden_state)

    premise_embedding, premise_attention = self.premise_embedder(processed_premise)
    processed_hypothesis, hidden_state = self.hypothesis_processor(hypothesis, hidden_state)
    hypothesis_embedding, hypothesis_attention = self.hypothesis_embedder(processed_hypothesis)
    combined = premise_embedding * hypothesis_embedding
    avg = torch.sum(combined, 1)/self.hp.attention_hops
    output = torch.sigmoid(self.linear_final(avg))
    return output, hypothesis_attention

##BAD DECLARE CODE

In [0]:
class BaselineDeclare(torch.nn.Module):
  def init_hidden(self):
    hidden_state = Variable(torch.zeros(2,self.hp.batch_size,self.hp.lstm_hidden_size)).cuda()
    cell_state = Variable(torch.zeros(2,self.hp.batch_size, self.hp.lstm_hidden_size)).cuda()
    return (hidden_state, cell_state)
  def reset_for_testing(self, new_batch):
    self.hp.batch_size = new_batch
    self.hidden_state = self.init_hidden()

  def load_embeddings(self, word_embeddings):
    self.premise_embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.premise_embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.premise_embedding_size = word_embeddings.size(1)
    self.hypothesis_processor = SequenceProcessor(word_embeddings, self.hp)
  
  def __init__(self, hp, word_embeddings):
    super(BaselineDeclare, self).__init__()
    self.hp = hp
    self.embeddings_size = word_embeddings.size(1)
    self.premise_embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.premise_embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.premise_embedding_size = word_embeddings.size(1)
    self.premise_lstm = torch.nn.LSTM(
        input_size = self.premise_embedding_size,
        hidden_size = hp.lstm_hidden_size,
        num_layers=1,
        batch_first=True,
        bidirectional=True
      )
    self.hypothesis_processor = SequenceProcessor(word_embeddings, hp)
    self.premise_linear = torch.nn.Linear(2*hp.lstm_hidden_size, 2*hp.lstm_hidden_size)

    self.linear_penultimate = torch.nn.Linear(101, 8)
    #TODO: add third dense layer with relu
    self.linear_final = torch.nn.Linear(8, hp.num_classes)

  def forward(self, premise, hypothesis):
    #get word embeddings for claim, take a mean over the length
    embeddings = self.premise_embeddings(premise)
    mean_embeddings = torch.unsqueeze(torch.sum(embeddings, 1) / self.hp.max_length, 1) #change to accurate size of lenfgth
  

    #get word embeddings for article, slap that onto the much smaller claim
    added_embeddings = self.premise_embeddings(hypothesis[:, :100])
    #TODO: use repeat function to get 100*100
    main_embeddings = torch.cat((mean_embeddings, added_embeddings), 1)
    #shape is 101 * 50

    #attention processing on claim+article combination
    processed_premise, hidden_state = self.premise_lstm(main_embeddings, self.hidden_state)
    attention_weights = softmax(self.premise_linear(processed_premise))#TODO: turn into row vector
    #simple embedding of article alone
    processed_hypothesis, hidden_state = self.hypothesis_processor(hypothesis, hidden_state)
    #matrix multiply of the two
    combined = torch.bmm(processed_hypothesis,attention_weights.transpose(1,2))
    #final processing - another average, and then a relu + sigmoid
    avg = torch.sum(combined, 1)/self.hp.max_length #todo: fix padding

    smaller = F.relu(self.linear_penultimate(avg))
    output = torch.sigmoid(self.linear_final(smaller))
    return output, torch.zeros(self.hp.batch_size, self.hp.attention_hops, self.hp.lstm_hidden_size*2).cuda()

## GOOD DECLARE CODE???


In [0]:
class RealDeclare(torch.nn.Module):
  def init_hidden(self):
    hidden_state = Variable(torch.zeros(2,self.hp.batch_size,self.hp.lstm_hidden_size)).cuda()
    cell_state = Variable(torch.zeros(2,self.hp.batch_size, self.hp.lstm_hidden_size)).cuda()
    return (hidden_state, cell_state)
  def reset_for_testing(self, new_batch):
    self.hp.batch_size = new_batch
    self.hidden_state = self.init_hidden()

  def __init__(self, hp, word_embeddings):
    super(RealDeclare, self).__init__()
    self.hp = hp
    self.embeddings_size = word_embeddings.size(1)
    self.premise_embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.premise_embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.premise_embedding_size = word_embeddings.size(1)
    self.hypothesis_processor = SequenceProcessor(word_embeddings, hp)
    self.premise_linear = torch.nn.Linear(2*hp.lstm_hidden_size, 1)

    self.linear_penultimate = torch.nn.Linear(100, 50)
    self.linear_almost_there = torch.nn.Linear(50, 8)
    #TODO: add third dense layer with relu
    self.linear_final = torch.nn.Linear(8, hp.num_classes)

  def forward(self, premise, hypothesis):
    #get word embeddings for claim, take a mean over the length
    embeddings = self.premise_embeddings(premise)
    lengths = self.hp.max_length - (premise == 0).sum(dim=1)
    lengths = lengths.repeat(50, 1).transpose(0,1)
    summed_embeddings = torch.sum(embeddings, 1)
    mean_embeddings = torch.unsqueeze(summed_embeddings / lengths, 1) #change to accurate size of lenfgth


    #get word embeddings for article, slap that onto the much smaller claim
    added_embeddings = self.premise_embeddings(hypothesis[:, :100])
    #TODO: use repeat function to get 100*100 #DONE!
    main_embeddings = torch.cat((mean_embeddings.repeat(1, 100, 1), added_embeddings), 2)
    #shape is 101 * 50

    #attention processing on claim+article combination
    attention_weights = softmax(self.premise_linear(main_embeddings.transpose(1,2)))#TODO: turn into row vector
    repeated_weights = attention_weights.repeat(1, 1, 100)
    #simple embedding of article alone
    processed_hypothesis, hidden_state = self.hypothesis_processor(hypothesis, self.hidden_state)
    #matrix multiply of the two
    combined = torch.bmm(processed_hypothesis, repeated_weights)

    #final processing - another average, and then a relu + sigmoid
    new_lengths = lengths.repeat(1, 2)
    avg = torch.sum(combined, 1)/new_lengths #todo: fix padding

    smaller = F.relu(self.linear_penultimate(avg))
    even_smaller = F.relu(self.linear_almost_there(smaller))
    output = torch.sigmoid(self.linear_final(even_smaller))
    return output, torch.zeros(self.hp.batch_size, self.hp.attention_hops, self.hp.lstm_hidden_size*2).cuda()

##TRAIN/TEST/HELPERS
HELPER FUNCTIONS FOR DOIN SOME TRAININ AND TESTIN

In [0]:
from inspect import signature
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def l2_matrix_norm(m):
  return torch.sum(torch.sum(torch.sum(m**2,1),1)**0.5).type(torch.DoubleTensor)


def load_data(data):
  for i in range(len(data)):
    data[i] = Variable(data[i]).cuda()
  return data

def free_data(data):
  for point in data:
    del(point)
def check_data(loader, model):
  sample_data = loader.dataset[0]
  print(torch.max(loader.dataset[:][-1]))
  model_params = len(signature(model).parameters)
  return len(sample_data) - 1 != model_params       

TRAIN FUNCT, ITS BIG CAUSE IT DOES PRETTY MUCH EVERYTHING

INCLUDING NORMALISATION IN THE WEIRD WAY THE SELF ATTENTIVE MODEL REQUIRES

ALSO A SWITCH TO ENSURE IT DOES THE BEST AT GETTING BOTH BINARY AND NON BINARY LOSS :)

In [0]:
def train(model=None, 
          train_loader=None, 
          loss_function=None, 
          optimiser=None, 
          hp=None, 
          using_gradient_clipping=False):
  
  model.reset_for_testing(train_loader.batch_size)
  model.train()
  losses = []
  accuracies = []
  is_binary = hp.num_classes == 1
  
  if train_loader.name == "entailment_data" and hp.num_classes != 3:
      raise ValueError("Three classes are needed for entailment to safely happen")
  elif train_loader.name == "fact_data" and hp.num_classes !=1:
      raise ValueError("Two classes are needed for fact checking to safely happen")
  torch.enable_grad()
  
  for epoch in range(hp.epochs):
    print("Running EPOCH:",epoch+1)
    total_loss = 0
    batch_count = 0
    correct = 0
    penal = 0
    for batch_index, train_data in enumerate(train_loader):
      #setting everything up
      model.hidden_state = model.init_hidden()
      train_data = load_data(train_data)
      
      #get y values - do forward pass and process
      predicted_y, attention = model(*train_data[:-1])
      actual_y = train_data[-1]
      squeezed_y = predicted_y.double().squeeze(1)

      #handling regularisation
      if hp.C > 0:
        attentionT = attention.transpose(1,2)
        identity = torch.eye(attention.size(1))
        identity = Variable(identity.unsqueeze(0).expand(train_loader.batch_size,
                                                         attention.size(1),
                                                         attention.size(1))).cuda()
        penal = l2_matrix_norm(attention@attentionT - identity).cuda()

      #get loss, accuracy
      if is_binary:
        loss = loss_function(squeezed_y, actual_y.double())
        loss += hp.C * penal/train_loader.batch_size
        correct += torch.eq(torch.round(squeezed_y), actual_y).data.sum()
      else:
        loss = loss_function(squeezed_y,actual_y.long()) + hp.C * (penal/train_loader.batch_size)
        correct += torch.eq(torch.argmax(squeezed_y, 1), actual_y).data.sum()
      total_loss += loss.data

      #cleaning up regularisation
      if hp.C > 0:
        del(penal)
        del(identity)
        del(attentionT)
      #woah we gotta do this to do backprop!!!
      optimiser.zero_grad()
      loss.backward()

      if hp.is_debug and batch_index % 10 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_index * len(train_data[0]), len(train_loader.dataset),
            100. * batch_index / len(train_loader), loss.item()
        ))

      if using_gradient_clipping:
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
      batch_count += 1
      optimiser.step()
      free_data(train_data)

    print("Average loss is:",total_loss/batch_count)
    correct_but_numpy = correct.data.cpu().numpy().astype(int)
    accuracy = correct_but_numpy / float(batch_count * train_loader.batch_size)
    print("Accuracy of the model", accuracy)
    losses.append(total_loss/batch_count)
    accuracies.append(accuracy)
  return losses, accuracies

TEST FUNCTION

THIS STRONG BOY GOES THROUGHH AND ADDS RESULTS ALL OVER THE SHOP

In [0]:
def batch_wise_evaluate(model, test_loader, hp):
  batch_count = 0
  total_accuracy = 0
  all_results = []
  model.eval()
  is_binary = hp.num_classes == 1
  real_results = []
  with torch.no_grad():
    for batch_index, test_data in enumerate(test_loader):
      #reset everything
      model.reset_for_testing(test_data[0].shape[0])
      test_data = load_data(test_data)
    
      #get ys from model and data
      y_predicted, _ = model(*test_data[:-1])
      y_actual = test_data[-1]
      y_squeezed = y_predicted.double().squeeze(1)

      #get accuracy
      if is_binary:
        total_accuracy += torch.eq(torch.round(y_squeezed), y_actual).data.sum()
        all_results.append(torch.round(y_squeezed))

      else: 
        total_accuracy += torch.eq(torch.argmax(y_squeezed,1), y_actual).data.sum()
        all_results.append(torch.argmax(y_squeezed, 1))

      batch_count += 1
      real_results.append(y_squeezed)
  return torch.cat(real_results, 0), torch.cat(all_results, 0)

In [0]:

def plot_stuff(epochs, losses, accuracies=None, title="sup nerds"):

  fig = plt.figure()
  if accuracies:
    plt.plot(range(1, epochs+1), accuracies, scalex=True, scaley=True, label="Accuracy")
    plt.annotate(str(accuracies[-1]), xy=(epochs,accuracies[-1]), xytext=(3, 3),textcoords="offset points")

  plt.plot(range(1, epochs+1), losses,scalex=True, scaley=True, label="Loss")
  plt.annotate(str(losses[-1]), xy=(epochs,losses[-1]), xytext=(3, 3),textcoords="offset points")
  plt.legend()
  plt.xlabel("Epochs", fontsize=16)
  plt.ylabel("Amount", fontsize=16)
  plt.title(title)
  plt.close()

NEW FUNCTIONS TO AUTOMATE THE RUNNING OF LOTS OF DATASETS/MODELS

In [0]:
def run_model(model, dataset, hp):
  runnable_model = model(hp, dataset.word_embeddings_small).cuda()
  bce_loss = torch.nn.BCELoss()
  cross_entropy_loss = torch.nn.CrossEntropyLoss()
  optimiser = torch.optim.Adam(runnable_model.parameters(), lr=0.01)
  losses, accuracies = train(model=runnable_model,
                       train_loader=dataset.train_loader,
                       loss_function=bce_loss,
                       optimiser = optimiser,
                       hp = hp,
                       using_gradient_clipping=True)
  plot_stuff(hp.epochs, losses, accuracies)
  torch.cuda.empty_cache()
  results, predicted_ys = batch_wise_evaluate(runnable_model, 
         dataset.test_loader,
         hp)
  return results, predicted_ys

def do_cross_eval(model, folds, hp):
  all_results = []
  for fold in folds:
    all_results.append(run_model())

def get_results(model_name, dataset_name, predictions, unnormalised_predictions, true_labels):
  precision = precision_score(predictions, true_labels, average='macro')
  recall = recall_score(predictions, true_labels, average='macro')
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1, average='macro') #1 means f_1 measure
  if (len(unnormalised_predictions.shape) == 1):
    fpr, tpr, thresholds = roc_curve(true_labels, predictions)
    auc_full = auc(fpr, tpr)
  else:
    auc_full = 0
  return {"model_name":model_name,
                  "dataset_name": dataset_name,
                  "precision":precision,
                  "recall": recall,
                  "accuracy": accuracy,
                  "f1": f1,
                  "auc": auc_full}

#RUNNING THE MODELS

In [0]:

datasets = {
    "politifact": fact_dataset,
    "snopes": snopes_dataset
}
models = {
    "my_model": TextualEntailmentModel,
    "sheena_model": BaselineSentenceEntailment,
    "broke_declare": BaselineDeclare,
    "real_declare": RealDeclare
}

##TextualEntailment Model

In [0]:
class Hyperparameters:
  lstm_hidden_size = 50
  dense_dimension = 20
  attention_hops = 10
  batch_size = BATCH_SIZE
  max_length = MAX_LENGTH
  gravity = 20
  num_classes = 1
  avg=True
  epochs = 3
  dropout=0.3
  C = 0.3
  is_debug = True


##CROSS VALIDATION ZONE

In [107]:
results_collector = []
for fold in snopes_folded_datasets:
  real_results, predicted_ys = run_model(models["my_model"], fold, Hyperparameters)
  evaluation_summary("textual entailment model", predicted_ys.cpu(), real_results.cpu(), fold.test_data)
  results_collector.append((real_results, predicted_ys))

Running EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/13291 (0%)]	Loss: 1.639968


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Train Epoch: 0 [2560/13291 (20%)]	Loss: 1.637590
Train Epoch: 0 [5120/13291 (39%)]	Loss: 1.639982
Train Epoch: 0 [7680/13291 (59%)]	Loss: 1.576631
Train Epoch: 0 [10240/13291 (78%)]	Loss: 1.358812
Train Epoch: 0 [12800/13291 (98%)]	Loss: 1.202754
Average loss is: tensor(1.5205, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.6540287990196079
Running EPOCH: 2
Train Epoch: 1 [0/13291 (0%)]	Loss: 1.112143
Train Epoch: 1 [2560/13291 (20%)]	Loss: 1.018874
Train Epoch: 1 [5120/13291 (39%)]	Loss: 0.990871
Train Epoch: 1 [7680/13291 (59%)]	Loss: 0.923158
Train Epoch: 1 [10240/13291 (78%)]	Loss: 0.943300
Train Epoch: 1 [12800/13291 (98%)]	Loss: 0.906545
Average loss is: tensor(0.9859, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.9742647058823529
Running EPOCH: 3
Train Epoch: 2 [0/13291 (0%)]	Loss: 0.903954
Train Epoch: 2 [2560/13291 (20%)]	Loss: 0.872605
Train Epoch: 2 [5120/13291 (39%)]	Loss: 0.840177
Train Epoch: 2 [7680/13291 (59%)]	Loss: 0.811531
Train Epoch: 

runnin my textual entailent model :)

In [75]:

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

real_results, predicted_ys = run_model(models["my_model"], datasets["politifact"], Hyperparameters)

#textual_entailment_model.to(device)



Running EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/23454 (0%)]	Loss: 1.644113


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


KeyboardInterrupt: ignored

In [0]:
evaluation_summary("textual entailment model", predicted_ys.cpu(), real_results.cpu(), datasets["politifact"].test_data)

running sheena's model

In [0]:
sheena_real_results, sheena_predicted_ys = run_model(models["sheena_model"], datasets["politifact"], Hyperparameters)

In [39]:
evaluation_summary("sheena model", sheena_predicted_ys.cpu(), sheena_real_results.cpu(), datasets["politifact"].test_data)


Evaluation for: sheena model
Classifier 'sheena model' has Acc=0.622 P=0.622 R=0.622 F1=0.622 AUC=0.622
              precision    recall  f1-score   support

         0.0      0.636     0.599     0.617      3105
         1.0      0.608     0.646     0.626      2997

    accuracy                          0.622      6102
   macro avg      0.622     0.622     0.622      6102
weighted avg      0.623     0.622     0.622      6102


Confusion matrix:
 [[1859 1062]
 [1246 1935]]


(0.6223625792521735, 0.62217870043957, 0.6217633562766306, 0.6217046730811568)

OK TESTING ON BROKE DECLARE

In [40]:
broke_real_results, broke_predicted_ys = run_model(models["broke_declare"], datasets["politifact"], Hyperparameters)

Running EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/23454 (0%)]	Loss: 1.646909


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Train Epoch: 0 [2560/23454 (11%)]	Loss: 1.639336
Train Epoch: 0 [5120/23454 (22%)]	Loss: 1.626918
Train Epoch: 0 [7680/23454 (33%)]	Loss: 1.556427
Train Epoch: 0 [10240/23454 (44%)]	Loss: 1.501157
Train Epoch: 0 [12800/23454 (55%)]	Loss: 1.458227
Train Epoch: 0 [15360/23454 (66%)]	Loss: 1.442755
Train Epoch: 0 [17920/23454 (77%)]	Loss: 1.445636
Train Epoch: 0 [20480/23454 (88%)]	Loss: 1.361013
Train Epoch: 0 [23040/23454 (99%)]	Loss: 1.264397
Average loss is: tensor(1.4964, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.7025669642857143
Running EPOCH: 2
Train Epoch: 1 [0/23454 (0%)]	Loss: 1.208051
Train Epoch: 1 [2560/23454 (11%)]	Loss: 1.182681
Train Epoch: 1 [5120/23454 (22%)]	Loss: 1.227636
Train Epoch: 1 [7680/23454 (33%)]	Loss: 1.140428
Train Epoch: 1 [10240/23454 (44%)]	Loss: 1.170687
Train Epoch: 1 [12800/23454 (55%)]	Loss: 1.171650
Train Epoch: 1 [15360/23454 (66%)]	Loss: 1.113836
Train Epoch: 1 [17920/23454 (77%)]	Loss: 1.159310
Train Epoch: 1 [20480/23454 (88%)

In [41]:
evaluation_summary("broke declare model", broke_predicted_ys.cpu(), broke_real_results.cpu(), datasets["politifact"].test_data)


Evaluation for: broke declare model
Classifier 'broke declare model' has Acc=0.633 P=0.631 R=0.632 F1=0.631 AUC=0.631
              precision    recall  f1-score   support

         0.0      0.585     0.624     0.604      2739
         1.0      0.676     0.640     0.657      3363

    accuracy                          0.633      6102
   macro avg      0.631     0.632     0.631      6102
weighted avg      0.635     0.633     0.633      6102


Confusion matrix:
 [[1709 1212]
 [1030 2151]]


(0.6306380284944597,
 0.6317789200757291,
 0.6325794821370042,
 0.6306415069073065)

TESTING ON REAL DECLARE

In [103]:
declare_real_results, declare_predicted_ys =  run_model(models["real_declare"], datasets["snopes"], Hyperparameters)

Running EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/11721 (0%)]	Loss: 1.639971


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Train Epoch: 0 [2560/11721 (22%)]	Loss: 1.642742
Train Epoch: 0 [5120/11721 (44%)]	Loss: 1.643826
Train Epoch: 0 [7680/11721 (67%)]	Loss: 1.626425
Train Epoch: 0 [10240/11721 (89%)]	Loss: 1.608865
Average loss is: tensor(1.6446, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.5249131944444444
Running EPOCH: 2
Train Epoch: 1 [0/11721 (0%)]	Loss: 1.717351
Train Epoch: 1 [2560/11721 (22%)]	Loss: 1.505505
Train Epoch: 1 [5120/11721 (44%)]	Loss: 1.434764
Train Epoch: 1 [7680/11721 (67%)]	Loss: 1.347603
Train Epoch: 1 [10240/11721 (89%)]	Loss: 1.342587
Average loss is: tensor(1.4397, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.8208333333333333
Running EPOCH: 3
Train Epoch: 2 [0/11721 (0%)]	Loss: 1.308084
Train Epoch: 2 [2560/11721 (22%)]	Loss: 1.212712
Train Epoch: 2 [5120/11721 (44%)]	Loss: 1.200435
Train Epoch: 2 [7680/11721 (67%)]	Loss: 1.232354
Train Epoch: 2 [10240/11721 (89%)]	Loss: 1.303127
Average loss is: tensor(1.2129, device='cuda:0', dtype=torch.fl

In [105]:
evaluation_summary("real declare model", declare_predicted_ys.cpu(), declare_real_results.cpu(), datasets["snopes"].test_data)


Evaluation for: real declare model
Classifier 'real declare model' has Acc=0.700 P=0.694 R=0.695 F1=0.695 AUC=0.694
              precision    recall  f1-score   support

         0.0      0.744     0.727     0.735      1749
         1.0      0.644     0.664     0.654      1301

    accuracy                          0.700      3050
   macro avg      0.694     0.695     0.695      3050
weighted avg      0.701     0.700     0.701      3050


Confusion matrix:
 [[1271  437]
 [ 478  864]]


(0.6939802001277411, 0.6954027534785443, 0.7, 0.6945610692349038)

                  "model_name":model_name,
                  "dataset_name": dataset_name,
                  "precision":precision,
                  "recall": recall,
                  "accuracy": accuracy,
                  "f1": f1,
                  "auc": auc

##VALIDATION LAND

In [101]:
avg_amount = 5
per_avg_amount = 10
import csv

full_results = []
avg_results = []
processed_results = []

def list_to_dict(results):
  big_results = {"model_name": [],
                 "dataset_name": [],
                "precision": [],
                 "recall": [],
                 "accuracy": [],
                 "f1": [],
                 "auc": []}
  for result in results:
    for key in result:
      big_results[key].append(result[key])

  return pd.DataFrame.from_dict(big_results)

def process_results(big_results):
  return (big_results["model_name"][0], big_results["dataset_name"][0], big_results.mean(), big_results.std())
  
  
def get_avgs(some_results):
  avg_results = {"model_name": some_results[0]["model_name"],
                 "dataset_name": some_results[0]["dataset_name"],
                "precision": 0.0,
                 "recall": 0.0,
                 "accuracy": 0.0,
                 "f1": 0.0,
                 "auc": 0.0}
  for result in some_results:
    for key in result:
      if type(result[key]) is float:
        avg_results[key] += result[key]
  
  for key in avg_results:
    if(type(avg_results[key] is float)):
      avg_results[key] /= len(some_results)
  
  return avg_results



for data_name in datasets:
  for model_name in models:
    some_results = []
  
    for per_avg_i in range(per_avg_amount):
      results, predicted_ys = run_model(models[model_name], datasets[data_name], Hyperparameters)
      full_results.append(get_results(model_name, data_name, predicted_ys.cpu(), results.cpu(), datasets[data_name].test_data))
      some_results.append(get_results(model_name, data_name, predicted_ys.cpu(), results.cpu(), datasets[data_name].test_data))
      
    processed_results.append(process_results(list_to_dict(some_results)))
    print(processed_results)
      #avg_results.append(get_avgs(some_results))

Running EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/23454 (0%)]	Loss: 1.640231


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Train Epoch: 0 [2560/23454 (11%)]	Loss: 1.632861
Train Epoch: 0 [5120/23454 (22%)]	Loss: 1.584366
Train Epoch: 0 [7680/23454 (33%)]	Loss: 1.558179
Train Epoch: 0 [10240/23454 (44%)]	Loss: 1.415853
Train Epoch: 0 [12800/23454 (55%)]	Loss: 1.231022
Train Epoch: 0 [15360/23454 (66%)]	Loss: 1.178902
Train Epoch: 0 [17920/23454 (77%)]	Loss: 1.105380
Train Epoch: 0 [20480/23454 (88%)]	Loss: 1.067210
Train Epoch: 0 [23040/23454 (99%)]	Loss: 1.103633
Average loss is: tensor(1.3588, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.7633499313186813
Running EPOCH: 2
Train Epoch: 1 [0/23454 (0%)]	Loss: 0.963361
Train Epoch: 1 [2560/23454 (11%)]	Loss: 0.929948
Train Epoch: 1 [5120/23454 (22%)]	Loss: 0.943169
Train Epoch: 1 [7680/23454 (33%)]	Loss: 0.978601
Train Epoch: 1 [10240/23454 (44%)]	Loss: 0.925199
Train Epoch: 1 [12800/23454 (55%)]	Loss: 0.912593
Train Epoch: 1 [15360/23454 (66%)]	Loss: 0.868571
Train Epoch: 1 [17920/23454 (77%)]	Loss: 0.894872
Train Epoch: 1 [20480/23454 (88%)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running EPOCH: 1
Train Epoch: 0 [0/23454 (0%)]	Loss: 1.640791
Train Epoch: 0 [2560/23454 (11%)]	Loss: 1.638111
Train Epoch: 0 [5120/23454 (22%)]	Loss: 1.636649
Train Epoch: 0 [7680/23454 (33%)]	Loss: 1.613779
Train Epoch: 0 [10240/23454 (44%)]	Loss: 1.600282
Train Epoch: 0 [12800/23454 (55%)]	Loss: 1.551470
Train Epoch: 0 [15360/23454 (66%)]	Loss: 1.480122
Train Epoch: 0 [17920/23454 (77%)]	Loss: 1.403122
Train Epoch: 0 [20480/23454 (88%)]	Loss: 1.373569
Train Epoch: 0 [23040/23454 (99%)]	Loss: 1.337759
Average loss is: tensor(1.5296, device='cuda:0', dtype=torch.float64)
Accuracy of the model 0.6760817307692307
Running EPOCH: 2
Train Epoch: 1 [0/23454 (0%)]	Loss: 1.220479
Train Epoch: 1 [2560/23454 (11%)]	Loss: 1.201358
Train Epoch: 1 [5120/23454 (22%)]	Loss: 1.232560
Train Epoch: 1 [7680/23454 (33%)]	Loss: 1.233302
Train Epoch: 1 [10240/23454 (44%)]	Loss: 1.135679
Train Epoch: 1 [12800/23454 (55%)]	Loss: 1.157284
Train Epoch: 1 [15360/23454 (66%)]	Loss: 1.221421
Train Epoch: 1 [17920

In [102]:
for result in full_results:
  print(result)

{'model_name': 'my_model', 'dataset_name': 'politifact', 'precision': 0.6159650423533861, 'recall': 0.6166411105620413, 'accuracy': 0.6138970829236315, 'f1': 0.6137051929513584, 'auc': 0.6159650423533862}
{'model_name': 'my_model', 'dataset_name': 'politifact', 'precision': 0.6018810764573677, 'recall': 0.6028130181590461, 'accuracy': 0.5994755817764668, 'f1': 0.599095030895942, 'auc': 0.6018810764573677}
{'model_name': 'my_model', 'dataset_name': 'politifact', 'precision': 0.6126830275748218, 'recall': 0.6125150125043307, 'accuracy': 0.612094395280236, 'f1': 0.612035785541073, 'auc': 0.6126830275748218}
{'model_name': 'my_model', 'dataset_name': 'politifact', 'precision': 0.6004185885878162, 'recall': 0.6002776859191488, 'accuracy': 0.599803343166175, 'f1': 0.599753638082202, 'auc': 0.6004185885878162}
{'model_name': 'my_model', 'dataset_name': 'politifact', 'precision': 0.6235426645777775, 'recall': 0.623320277463816, 'accuracy': 0.6234021632251721, 'f1': 0.6231981091109211, 'auc': 0